# Uncertainty propagation

In [7]:
import clifford as cf
from clifford.tools.g3 import *
import numpy as np

In [8]:
layout, blades = cf.Cl(2)
e1 = blades['e1']; e2 = blades['e2']; e12 = blades['e12']

In [28]:
def propagateGeometricProduct(a, b, E_aa, E_bb, E_ab = None, E_ba = None):
    """Build covariance matrix following Ch 5. Eq 5.64.
       While building matrix, also care of building Eq 5.73, 5.74, 5.75"""
    
    # Prepare matrices
    a_m = layout.get_left_gmt_matrix(a)
    b_m = layout.get_right_gmt_matrix(b)
    E_uu = np.zeros_like(a_m)
    E_ua = np.zeros_like(a_m)
    E_ub = np.zeros_like(a_m)
    
    # Compute part of covariances related to variance of a and b 
    E_ua = np.matmul(b_m, E_aa)
    E_ub = np.matmul(a_m, E_bb)
    E_uu = np.matmul(E_ua, b_m.T) + np.matmul(E_ub, a_m.T)
     
    # Add part of covariances related to cross-covariance of a and b
    if E_ab is not None:
        Gr_Eab = np.matmul(b_m, E_ab)
        E_ub += Gr_Eab
        Euu += np.matmul(Gr_Eab, a_m.T)
    if E_ba is not None:
        Gl_Eba = np.matmul(a_m, Eba)
        E_ua += Gl_Eba
        Euu += np.matmul(Gl_Eba, b_m.T)
    
    output = [E_uu, E_ua, E_ub]
    return output

### Define some random multivectors

In [29]:
# Multivectors a,b and their covariance matrices E_aa, E_bb
# a,b are uncorrelated: E_ab = E_ba = 0
a = 0.0 + 1.0*e1 + 2.0*e2 + 3.0*e12
b = 3.0 + 2.0*e1 + 1.0*e2 + 0.0*e12
E_aa = 4*np.eye(4, dtype=np.float64)
E_bb = 5*np.eye(4, dtype=np.float64)
print(E_aa,"\n", E_bb)
E_ab = np.zeros((4,4))
E_ba = np.zeros((4,4))
c = a * b
print("c: ", c)
print("c coeffs: ", c.value)

[[4. 0. 0. 0.]
 [0. 4. 0. 0.]
 [0. 0. 4. 0.]
 [0. 0. 0. 4.]] 
 [[5. 0. 0. 0.]
 [0. 5. 0. 0.]
 [0. 0. 5. 0.]
 [0. 0. 0. 5.]]
c:  4.0 + (6.0^e1) + (6.0^e12)
c coeffs:  [4. 6. 0. 6.]


In [31]:
E_cc = propagateGeometricProduct(a, b, E_aa, E_bb)
print("C covariance:\n", E_cc[0])
print(E_cc[1])
print(E_cc[2])

C covariance:
 [[126. 108.  -6.   0.]
 [108. 126.   0.  54.]
 [ -6.   0. 126.  12.]
 [  0.  54.  12. 126.]]
[[12.  8.  4.  0.]
 [ 8. 12.  0.  4.]
 [ 4.  0. 12. -8.]
 [ 0.  4. -8. 12.]]
[[  0.   5.  10. -15.]
 [  5.   0.  15. -10.]
 [ 10. -15.   0.   5.]
 [ 15. -10.   5.   0.]]


In [12]:
(a_m.shape)

NameError: name 'a_m' is not defined